In [ ]:
#### REQUIREMENTS IF USING SELENIUM TO WEBSCRAPE - IGNORE ####

# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
# from selenium import webdriver
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
#### O2 Web scraper built using BeautifulSoup and pandas ####
#### Generates a pandas dataframe and writes a JSON file locally ####

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
#### ACTUAL WEB SCRAPE MODULE ####
## This block will dump the web scrape data into a pandas dataframe 'final_df' ##

baseurl = 'https://www.o2.co.uk'
contract_types = ['paymonthly','payasyougo']

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
}

productlinks = []
productset = []
for ct in contract_types:
  r = requests.get(f'https://www.o2.co.uk/shop/phones?contractType={ct}', headers=headers)

  soup = BeautifulSoup(r.content, 'lxml')

  productlist = soup.find_all('div', class_="device-tile-wrapper")
  for productgroup in productlist:
    for item in productgroup.find_all('a', href=True):
      brand = item.find('span', class_='brand-name').text.strip()
      model = item.find('div', class_='model-family').text.strip()
      link = (baseurl + item['href'])
      rating = item.find('span', itemprop='ratingValue').text.strip()
      reviewcount = item.find('span', itemprop='bestRating').text.strip()
      price = item.find

      if ct == 'paymonthly':
        upfront_cost = item.find('span', class_='headline').text.strip()
        mp_pounds = item.find('span', class_='costVal').text.strip()
        mp_pence = item.find('span', class_='costPence').text.replace('*','').strip()
        monthly_price = mp_pounds + mp_pence
        contract_type = 'Pay Monthly'
      else:
        upfront_cost = item.find('span', class_='headline').text.strip()
        monthly_price = 'N/A'
        contract_type = 'Pay As You Go'
        

      productlinks.append(link)
      product = {
          'brand': brand,
          'model': model,
          'upfront_cost': upfront_cost,
          'monthly_price': monthly_price,
          'link': link,
          'rating': round(float(rating),2),
          'reviews': reviewcount + ' reviews',
          'contract_type': contract_type
      }
      productset.append(product)

spec_data = []
for link in productlinks:
  index = productlinks.index(link)
  total = len(productlinks)
  print('Extracting spec data ' + str(index) + ' out of ' + str(total) + '...')
  df = pd.read_html(link)

  spec_table = df[0].transpose() # transpose the spec data
  new_header = spec_table.iloc[0] #grab the first row for the header
  spec_table = spec_table[1:] #take the data less the header row
  spec_table.columns = new_header #set the header row as the df header
  spec_table['link'] = link # add the link as a column
  spec_table = spec_table.loc[:,~spec_table.columns.duplicated()] # rudimentary handling of cases with duplicate columns
  spec_data.append(spec_table)

productset_df = pd.DataFrame(productset)
spec_concat = pd.concat(spec_data)

## Exports data into pandas dataframe ##

final_df = pd.merge(productset_df, spec_concat, how = 'left', on = 'link')

In [22]:
## Export dataframe to CSV (locally) ##

from datetime import datetime

today_date = datetime.today().strftime('%Y-%m-%d')
upload_prefix = 'O2_Product_Webscrape'

upload_name = '_'.join([upload_prefix, today_date]) + '.csv'

print(upload_name)

final_df.to_csv(upload_name, index=False)

O2_Product_Webscrape_2022-03-05.csv


In [ ]:
!gcloud auth login

In [ ]:
!gsutil cp O2_Product_Webscrape_2022-03-05.csv gs://test_dt/test_webscrape/

In [ ]:
#######################################################

In [1]:
import requests
from bs4 import BeautifulSoup

r = requests.get(f'https://www.o2.co.uk/shop/phones?contractType=payasyougo')
soup = BeautifulSoup(r.content, 'lxml')

all_products_test = soup.find_all('div', class_="details")

In [ ]:
print(all_products_test)